In [27]:
from pyrosm import get_data, OSM
import numpy as np
import pandas as pd
import geopandas as gpd

In [28]:
class Graph:
    def __init__(self, vertices : list, edges : list, neighbours : dict, weights : dict, positions : dict):
        self.vertices = vertices
        self.edges = edges
        self.neighbours = neighbours
        self.weights = weights
        self.positions = positions

In [ ]:
class Heap:
    def __init__(self):
        self.heap = []
    
    def is_empty(self):
        return len(self.heap) == 0
    
    def add_with_priority(self, k, w):
        inserted = False
        for i in range(len(self.heap)):
            cur_k, cur_w = self.heap[i]
            if w < cur_w:
                self.heap.insert(i, (k, w))
                inserted = True
        
        if not inserted:
            self.heap.append(k, w)
    
    def decrease_priority(self, k, w):
        for i in range(len(self.heap)):
            

In [22]:
def Dijkstra(G : Graph, source, target):
    Q = []
    dist = {}
    prev = {}

    for v in G.vertices:
        dist[v] = np.inf
        prev[v] = np.nan
        Q.append(v)

    dist[source] = 0

    while len(Q) > 0:
        print(len(Q))
        dist = {k: v for k, v in sorted(dist.items(), key=lambda item: item[1])}
        keys = list(dist.keys())
        u = keys[0]
        for v in keys:
            if v in Q:
                u = v
                break
        
        Q.remove(u)
        if u == target:
            break

        if u in G.neighbours.keys():
            for v in G.neighbours[u]:
                if v in Q:
                    alt = dist[u] + G.weights[(u, v)]
                    if alt < dist[v]:
                        dist[v] = alt
                        prev[v] = u

In [4]:
fp = get_data("copenhagen")
osm = OSM(fp)

In [5]:
raw_vertices, raw_edges = osm.get_network("driving", nodes=True)

In [24]:
vertices = []
positions = {}
for i, lon, lat in zip(raw_vertices["id"], raw_vertices["lon"], raw_vertices["lat"]):
    vertices.append(i)
    positions[i] = (lon, lat)

edges = []
neighbours = {}
weights = {}
for u, v, length in zip(raw_edges["u"], raw_edges["v"], raw_edges["length"]):
    edges.append((u, v))
    weights[(u, v)] = length

    if u in neighbours.keys():
        neighbours[u] += [v]
    else:
        neighbours[u] = [v]

In [15]:
raw_edges

,access,area,bicycle,bicycle_road,bridge,busway,cycleway,foot,footway,highway,...,width,id,timestamp,version,tags,osm_type,geometry,u,v,length
0,None,None,None,None,None,None,None,None,None,residential,...,None,2796,0,-1,"{""source:maxspeed"":""DK:urban""}",way,"LINESTRING (12.54874 55.71641, 12.54901 55.71624)",25981317,422248375,24.760
1,None,None,None,None,None,None,None,None,None,residential,...,None,2796,0,-1,"{""source:maxspeed"":""DK:urban""}",way,"LINESTRING (12.54901 55.71624, 12.54994 55.71569)",422248375,1356013905,85.438
2,None,None,None,None,None,None,None,None,None,residential,...,None,2796,0,-1,"{""source:maxspeed"":""DK:urban""}",way,"LINESTRING (12.54994 55.71569, 12.55058 55.71530)",1356013905,4677457060,58.319
3,None,None,None,None,None,None,None,None,None,residential,...,None,2796,0,-1,"{""source:maxspeed"":""DK:urban""}",way,"LINESTRING (12.55058 55.71530, 12.55076 55.71520)",4677457060,1356013901,15.896
4,None,None,None,None,None,None,None,None,None,residential,...,None,2796,0,-1,"{""source:maxspeed"":""DK:urban""}",way,"LINESTRING (12.55076 55.71520, 12.55160 55.71470)",1356013901,687890,77.036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
264791,None,None,None,None,None,None,None,None,None,service,...,None,1028416375,0,-1,None,way,"LINESTRING (12.56831 55.76008, 12.56826 55.76010)",2090734999,9481351489,3.956
264792,None,None,None,None,None,None,None,None,None,service,...,None,1028416375,0,-1,None,way,"LINESTRING (12.56826 55.76010, 12.56780 55.76010)",9481351489,9481351488,28.611
264793,None,None,None,None,None,None,None,None,None,service,...,None,1028416381,0,-1,None,way,"LINESTRING (12.56873 55.75985, 12.56883 55.75986)",9481351512,9481351510,6.409
264794,None,None,None,None,None,None,None,None,None,service,...,None,1028416381,0,-1,None,way,"LINESTRING (12.56883 55.75986, 12.56916 55.75989)",9481351510,9481351511,20.774


In [25]:
source = 25981317
target = 687890
G = Graph(vertices, edges, neighbours, weights, positions)
Dijkstra(G, source, target)

246074
246073
246072
246071
246070
246069
246068
246067
246066
246065
246064
246063
246062
246061
246060
